## Etude code de triche, version finale (v1)
la version 3 n'a pas marché pour plusieurs raisons. J'ai donc décidé de sauter le pas et de passer du côté *pré-trained* de la force. En plus d'être celle qui sera finalement concluante, elle me parait plus à propos par rapport au projet principal.

  * **projet principal** : Je le rappelle au cas où, mais ce code dois servir à un projet collaboratif avec Asyl_himi. Tandis qu'il se farcit le côté *nouvelle*, moi je m'occupe de la partie *code*.
  
  * **mode opératoire** : J'ai utilisé 2 modèles pré-entrainés: *gpt-2* et *BERT*. Seul le modèle gpt-2 a été entrainé à écrire comme Balzac. L'entrainement, qui n'a pas été fait sur notebook, s'est bien passé. Malheureusement, lors de la génération de texte, il peine à faire preuve d'imagination après un point. C'est pour ça que j'utilise *BERT*! pour relancer une pièce dans la machine.
la (V2), si elle existe, n'aura pas de *BERT*!


In [ ]:
from random import randint
from transformers import  AutoTokenizer, AutoModelForCausalLM
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
import tensorflow.keras as keras
from transformers import Trainer, TrainingArguments
import re
#/content/drive/MyDrive/triche

  1.  On commence par mettre les fonctions qui vont nous servir par la suite.

In [ ]:
def script_file (adress,name,content):#script fichier
    #adress=l'adresse dans lequel se trouvera le fichier
    #name=nom du fichier (penser à ajouter .txt)
    #content= le contenue.

    f2=open(adress+name,"w",encoding="utf8")
    print(content,file=f2)
    f2.close()

In [ ]:
def bert_generator (input_text,suff_max_len=100):#generateur transformer BERT
  #input_text= le texte qui sera en input
  #suff_max_len= max length en plus

  #---importation du modèle et des tokens
  tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
  model = TFGPT2LMHeadModel.from_pretrained("gpt2")

  #---tokenizer l'input text
  input_ids = tokenizer(input_text, return_tensors="tf").input_ids

  #---générer l'output texte
  outputs = model.generate(input_ids,
                          max_length=len(input_ids)+suff_max_len,
                          num_return_sequences=1,
                          temperature=0.7,            # Contrôle la créativité
                          top_p=0.9,                  # Active nucleus sampling
                          top_k=50,                   # Limite aux 50 tokens les plus probable
                          repetition_penalty=1.2 )

  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  return generated_text



  2.  On continue par importer le modèle qu'on a entrainé plus tôt ainsi que les tokens associés.

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/triche/balzac_model")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/triche/balzac_model_token")

  3.  Et enfin on génère le texte.

In [ ]:
#model = AutoModelForCausalLM.from_pretrained("./balzac_model")
#tokenizer = AutoTokenizer.from_pretrained("./balzac_model_token")

# Texte initial
#prompt = "In a luxurious 19th-century Parisian salon, the great Prince de Morgade welcomed his cousins"
#prompt = "In a luxurious 19th-century Parisian salon,"
prompt="In a luxurious 19th-century Parisian salon, the dining"
current_text = prompt
stock=[prompt]

for _ in range(40):  # boucle qui permet de faire _ générations
    #---initialisation des données d'entrée
    print(_,"\n \n")
    input_tokens = tokenizer(current_text, return_tensors="pt", padding=True, truncation=True)

    #---génération de texte
    print("len input_token",len(input_tokens[0]))
    suff_max_len=120
    output_tokens = model.generate(
        input_tokens["input_ids"],
        attention_mask=input_tokens["attention_mask"],
        max_length=len(input_tokens[0])+suff_max_len,  #
        do_sample=True,
        temperature=0.9, #originalité
        top_k=50,
        top_p=0.7,
        repetition_penalty=1.1,
        no_repeat_ngram_size=3,  # fait pour éviter les répétitions directes
        pad_token_id=tokenizer.pad_token_id
    )


    print("len output_tokens[0]",len(output_tokens[0]))
    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    print("generated_text",generated_text)

    #---extraction et analyse de la partie générée (uniquement)
    new_text = generated_text[len(current_text):].strip()
    #---si new_text est vide, on s'aide d'un autre transformer (pas entrainé)= BERT
    if new_text=="":

      generated_text=bert_generator (generated_text,suff_max_len)

      #generated_text=bert_generator (generated_text)
      new_text = generated_text[len(current_text):].strip()
      print("\n"," ---"*20,"\n","new_text_bert=", new_text,"\n"," ---"*20,"/n")
      new_text = new_text.split()[0]
      print("\n","_____new new_text_bert=", new_text,"\n")

    #---ajout de ce qui a été généré dans stock + current_text
    stock.append(new_text)
    current_text += " " + new_text
    print("\n"," ***"*20,"\n","current_text=",current_text,"\n"," ***"*20,"/n")

    if _ %4 ==0 and _!=0:
      stock.append("\n")#paragraphe
      #re.split(r"[;.]", current_text)
      #--on désengorge le current_text pour éviter d'atteindre le max_len.
      if re.split(r"[;.]", current_text)[-1]=="" or re.split(r"[;.]", current_text)[-1]==" ":
        current_text = re.split(r"[;.]", current_text)[-2]
      else:
        current_text=re.split(r"[;.]", current_text)[-1]

#---résultat final
print("Texte généré complet :")
print(current_text)
print(" ".join(stock))


0 
 

len input_token 13
len output_tokens[0] 21
generated_text In a luxurious 19th-century Parisian salon, the dining room was divided into two rooms.

  *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** 
 current_text= In a luxurious 19th-century Parisian salon, the dining room was divided into two rooms. 
  *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** /n
1 
 

len input_token 20
len output_tokens[0] 21
generated_text In a luxurious 19th-century Parisian salon, the dining room was divided into two rooms.


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may obse


  --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 
 new_text_bert= The first had an open kitchen with three large tables and one small table in front of it; this served as both seating for guests or to sit on while they were eating dinner at their own place (the second is reserved only by staff).
The third floor also featured four separate bedrooms that could be shared between people who wanted more space but didn't want too much privacy from other diners: there are no windows so you can see through them without having your eyes closed when looking down onto something else's 
  --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- /n

 _____new new_text_bert= The 


  *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** 
 current_text= In a luxurious 19th-century Parisian salon, the dining room was divided into two rooms. The 
  *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



  --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 
 new_text_bert= The second floor consisted entirely on wood floors: it contained an open kitchen where guests could sit down from their chairs as they pleased without any needling them about anything else besides 
  --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- /n

 _____new new_text_bert= The 


  *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** 
 current_text= In a luxurious 19th-century Parisian salon, the dining room was divided into two rooms. The first had marble windows and covered with white plaster; there were no curtains in sight of each other for three hundred francs (or twelve thousand duchesse de nucingen's), while all manner outcasts came by candlelight to enjoy themselves at one table or another--a scene that seemed natural given its surroundings. The 
  *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** ***

...


  --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 
 new_text_bert= I am sure that you are not too much surprised at the fact, but if this is true then we shall be able to get along well together."
"I see," said he; "you must know how hard and difficult things can become when one man's life becomes another woman's business!" He was very glad indeed with his words as they were spoken—he had been so busy lately trying to convince himself 
  --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- /n

 _____new new_text_bert= I 


  *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** 
 current_text=  she has always made a terrible bargain for my happiness by making all sorts out of myself in exchange for your friendship! She will make no more ado about it than i have done to her. I 
  *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** /n
35 
 

len input_token 39
len output_toke

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



  --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 
 new_text_bert= The girl was silent as the door opened behind them when they entered into an open room with two large tables on either side which were covered with bookshelves filled with pictures from various periods before their time: "I am sorry," said one woman who had been sitting at this table since morning while another sat 
  --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- /n

 _____new new_text_bert= The 


  *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** 
 current_text=  she has always made a terrible bargain for my happiness by making all sorts out of myself in exchange for your friendship! She will make no more ado about it than i have done to her. I know that you are very fond of me, and we shall never be apart; but now--" The 
  *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** /n
37 
 

len inpu

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



  --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 
 new_text_bert= "I'm sorry," said the man who had been sitting beside him with a hand on his shoulder; then they were gone from sight for some time before it was clear that there would be no more of this sort ever coming up in any future conversation between us."
The old gentleman looked at me as if I might say something about my own past life or what happened when we met here last night—he did not seem very interested either way by anything which could have come out of our mouths except perhaps an idea 
  --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- /n

 _____new new_text_bert= "I'm 


  *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** 
 current_text=  but now--" The words seemed to reach his throat, and he opened them again. "I'm 
  *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** *** /n
39 
 

len input_token

In [ ]:
print(" ".join(stock))
print("------")
print(re.split(r"[;.]", " ".join(stock)))

In a luxurious 19th-century Parisian salon, the dining room was divided into two rooms. The first had marble windows and covered with white plaster; there were no curtains in sight of each other for three hundred francs (or twelve thousand duchesse de nucingen's), while all manner outcasts came by candlelight to enjoy themselves at one table or another--a scene that seemed natural given its surroundings. The second is decorated as if it belonged exclusively among those apartments where women are paid less than men: these furnishings served more besides their own needs on parisienne houses when they could afford them like this but only so far from luxury did mme. 
 . The third, which bears the title of " the delphine de la vauquer," was built in a hotel at  Vaudois. 
 . It is not known whether it belonged to her or that she had given up its furnishings for another house. The 
 rest of the family were at their lowest ebb, and they had to be left in a state where there was no prospect for

  4.  J'exporte le texte

In [ ]:
script_file ("/content/drive/MyDrive/triche/","generated_text.txt"," ".join(stock))

NameError: name 'script_file' is not defined

Et voilà. Il ne me resterait qu'à traduire en français.
Ce code sera utilisé par *Asyl_himi* pour sa nouvelle.


<center> <b>Post_face </b> </center>
J'ai beaucoup appris à travers ce projet, qui a été à la fois des hauts, mais surtout des bas. J'ai évidemment appris énormément, dû enchainer les trouvailles pour passer entre les mailles du filet du piège. De mon côté, je ne pense pas m'arrêter là. à vrai dire, je suis assez frustré par les versions loupés. Il y a sans-doute des erreurs bidons qui expliquent mes résultats pas ouf.